In [1]:
!pip install opensmile

     |████████████████████████████████| 4.5 MB 4.1 MB/s 
     |████████████████████████████████| 635 kB 65.4 MB/s 
     |████████████████████████████████| 47 kB 3.7 MB/s 
     |████████████████████████████████| 167 kB 56.1 MB/s 
     |████████████████████████████████| 596 kB 50.0 MB/s 
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=169061 sha256=c526f5a76d4192c15c1d4b44ef640786809e9ff60bb548b97eb3d1b246f226da
  Stored in directory: /root/.cache/pip/wheels/47/60/19/6d020fc92138ed1b113a18271e83ea4b5525fe770cb45b9a2e
Successfully built iso-639
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import opensmile
import IPython
import os
import matplotlib.pyplot as plt


smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)
y = smile.process_file('Filename')

In [ ]:
def process_audio(dir, set, lab):

  data = []

  result = pd.DataFrame()



  print(f'[>] Set: {set} ...')



  try:

    for sample in os.listdir(dir):

      if sample.split('.')[0] in lab[lab['Type'] == set].Filename.tolist():

        df = pd.read_csv(f"MOSI/opensmile_all/{sample}", delimiter=',', skiprows=range(0, 95))

        audio_info = list(df.columns)

        # Remove unwanted char
        for val in audio_info:

          if '.' in val[-4:len(val)]:

            audio_info[audio_info.index(val)] = val.replace('.', '')

        # To DataFrame

        df = pd.DataFrame(np.asarray(audio_info[1:-1]).reshape(1, -1), columns=[f"F{i}" for i in range(0, len(audio_info) - 2)])

        df.insert(0, 'sentiment', [1 if np.asarray(lab.loc[lab['Filename'] == sample.split('.')[0]]['Sentiment'])[0] == 'positive' else 0 for _ in range(0, len(df))])

        # Append to a list

        if len(df) != 1:

          print(len(df))

        data.append(df)

    result = pd.concat(data, ignore_index=True)
     except Exception as ex:

    print(f"{ex}")

    return -1



  # Save samples and labels

  X = result.iloc[:, result.columns != 'sentiment']

  y = result.iloc[:, result.columns == 'sentiment']

 

  X.to_csv(f'./MOSI/X_audio_{set}.csv')

  y.to_csv(f'./MOSI/y_audio_{set}.csv')



  return (X, y)